In [3]:
%pip install -qU \
  langchain==0.1.1 \
  langchain-community==0.0.13 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  pinecone-client==3.1.0 \

ERROR: Invalid requirement: ''

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [62]:
loader1 = CSVLoader("../output/technical_documents.csv")
doc = loader1.load()
doc

[Document(page_content='topic_name: Market Organization and Structure\nlearning_outcome: explain the main functions of the financial system\ntechnical_document: The main functions of the financial system include allowing strangers to contract with each other, moving money through time, hedging risks, and exchanging assets. Investors save money for the future, expecting a rate of return, while borrowers move money from the future to the present. Hedgers trade to reduce risks, and information-motivated traders identify undervalued instruments. Financial intermediaries connect buyers and sellers, creating instruments through various activities like securitization and managing investment funds. Clearinghouses, maintenance margins, and settlement guarantees manage credit risk in markets. Margin loans allow for larger positions but expose investors to more risk. Orders are instructions to trade, with market orders filling quickly but at potentially inferior prices, while limit orders may fil

In [63]:
# Assuming `documents` is a list of Document objects
list_of_dictionaries = [
    {
        'page_content': docs.page_content,
        'metadata': {
            'source': docs.metadata['source'],
            'row': docs.metadata['row']
        }
    } for docs in doc
]

list_of_dictionaries

[{'page_content': 'topic_name: Market Organization and Structure\nlearning_outcome: explain the main functions of the financial system\ntechnical_document: The main functions of the financial system include allowing strangers to contract with each other, moving money through time, hedging risks, and exchanging assets. Investors save money for the future, expecting a rate of return, while borrowers move money from the future to the present. Hedgers trade to reduce risks, and information-motivated traders identify undervalued instruments. Financial intermediaries connect buyers and sellers, creating instruments through various activities like securitization and managing investment funds. Clearinghouses, maintenance margins, and settlement guarantees manage credit risk in markets. Margin loans allow for larger positions but expose investors to more risk. Orders are instructions to trade, with market orders filling quickly but at potentially inferior prices, while limit orders may fill at 

In [64]:
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [65]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function


def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)


tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")


26

In [66]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n", ":", "", "\n\n"]
)

In [24]:
chunks = text_splitter.split_text(list_of_dictionaries[6]['page_content'])[:3]
chunks

['topic_name: Market Organization and Structure\nlearning_outcome: compare execution, validity, and clearing instructions\ntechnical_document: The reading discusses the importance of understanding how the financial system operates, particularly in terms of execution, validity, and clearing instructions. Execution refers to the process of carrying out trading decisions, while validity ensures that contracts are honored by all parties involved. Clearing instructions, such as variation margins and settlement guarantees, help manage credit risk and facilitate trading between strangers in the market.',
 'One key aspect highlighted is the use of futures contracts, which are forward contracts guaranteed by clearinghouses. These contracts provide a level of security for traders, ensuring that transactions can be completed without concerns about counterparty risk. This makes futures contract markets attractive to hedgers and information-motivated traders.\n\nFurthermore, the reading discusses t

In [25]:
tiktoken_len(chunks[0]), tiktoken_len(chunks[1]), tiktoken_len(chunks[2])

(97, 118, 51)

In [67]:
import os
import configparser
global config
config = configparser.RawConfigParser()
config.read('../configuration.properties')

api_key = config['PINECONE']['PINECONE_API_KEY']
environment = config['PINECONE']['PINECONE_ENVIRONMENT']
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']


In [68]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [29]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

In [30]:
from json import load
from pinecone import Pinecone
import os

# configure client
pc = Pinecone(api_key=api_key)

/Users/riteshchoudhary/Downloads/Assignment5/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [69]:
from pinecone import PodSpec
spec = PodSpec(environment=environment)

In [70]:
index_name = 'assigmment4'

In [178]:
import time

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# we create a new index
pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )

# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [34]:
index = pc.Index(index_name)
# wait a moment for connection
import time
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00115,
 'namespaces': {'': {'vector_count': 115}},
 'total_vector_count': 115}

In [180]:
from langchain.embeddings.openai import OpenAIEmbeddings
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []

for i, record in enumerate(tqdm(list_of_dictionaries)):
    # first get metadata fields for this record
    metadata = {
        'source': record['metadata']['source'],
        'id': record['metadata']['row']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(
        record['page_content'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

100%|██████████| 31/31 [00:02<00:00, 13.20it/s]


In [71]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00115,
 'namespaces': {'': {'vector_count': 115}},
 'total_vector_count': 115}

In [72]:
import boto3
def aws_connection():
    try:
        # AWS details fetch
        config = configparser.ConfigParser()
        config.read('configuration.properties')

        # s3 connection details
        aws_access_key = config['AWS']['access_key']
        aws_secret_key = config['AWS']['secret_key']
        bucket_name = config['AWS']['bucket']
        txt_file_folder = config['AWS']['txt_file_folder_name']
        analysis_folder_name = config['AWS']['analysis_folder_name']
        question_sets_folder_name = config['AWS']['question_folder_name']

        s3_boto3_client = boto3.client(
            's3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

        return s3_boto3_client, bucket_name, analysis_folder_name, txt_file_folder, question_sets_folder_name

    except Exception as e:
        print("Exception in aws_connection function: ", e)
        return

In [73]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/Users/riteshchoudhary/Downloads/Assignment5/venv/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [53]:
query = "Question: Which of the following statements about equity securities is correct? Options: A. Equity securities have historically offered average real annual returns lower than government bills and bonds. B. Common shares do not give investors any claim on a company\'s operating performance.C. Preference shares are a form of equity in which payments made to common stockholders take precedence. D. Private equity securities are primarily traded in secondary equity markets."

response = vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

print(response)

[Document(page_content="Common shares give investors ownership in the company and the opportunity to participate in its performance and decision-making. Preference shares, on the other hand, provide priority in dividend payments over common shareholders. Private equity securities are issued to institutional investors in private placements and are not traded in secondary markets. These investments aim to increase long-term value creation by allowing companies to focus on their operational activities.\n\nThe cost of equity is the minimum rate of return that stockholders require from the company for investing in its equity. Understanding the characteristics of equity securities, such as their risk and return profiles, is essential for investors to make informed investment decisions. Additionally, a company's accounting return on equity reflects the total return it earns on shareholders' book equity.", metadata={'chunk': 1.0, 'id': 28.0, 'source': '../data/output/technical_documents.csv'})

In [82]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
import pandas as pd
# completion llm

def similarity_search(query):

    text_field = "text"  # the metadata field that contains our text
    index = "assigmment4"  # the name of the index we created
    embed = OpenAIEmbeddings(
        model='text-embedding-ada-002',
        openai_api_key=OPENAI_API_KEY
    )

    response = vectorstore.similarity_search(
        query,  # our search query
        k=3  # return 3 most relevant docs
    )

    for document in response:
        print(document.page_content)

    return document

def generate_openai_response(query, context):

    api_key = config['OPENAI']['OPENAI_API_KEY']
    print("attempting to connect to openai")

    llm = ChatOpenAI(
        openai_api_key=api_key,
        model_name='gpt-3.5-turbo',
        temperature=0.0,
        max_tokens=2048,
        frequency_penalty=0.1,
        presence_penalty=0.1,
    )

    messages = [
        SystemMessage(content="You are an AI chat assistant based on the provided CONTEXT below. You will be provided with a question having multiple answer choices, out of which only one is correct. You have to answer the Question strictly from the CONTEXT. Ensure that your responses are solely derived from the CONTEXT provided, without being open-ended.\n\nCONTEXT:\n" + context),
        HumanMessage(content=query)
    ]

    response = llm.invoke(messages)
    response = response.content

    return response


def qa_using_ks():

    df = pd.read_csv(
        '../output/qa_df_set_a_b_qa.csv')
    df = df.drop(columns=['Unnamed: 0'], axis=1)
    new_df = df.copy()

    # Use a lambda function to apply 'generate_openai_response' to both 'question' and 'answer' columns
    new_df['openai_response'] = new_df.apply(lambda row: generate_openai_response(
        row['question'], similarity_search(row['question'])), axis=1)

    new_df.to_csv(
        '/output/qa_df_set_a_b_qa_with_openai_response.csv', index=False)

    return new_df

qa_using_ks()

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


Common shares give investors ownership in the company and the opportunity to participate in its performance and decision-making. Preference shares, on the other hand, provide priority in dividend payments over common shareholders. Private equity securities are issued to institutional investors in private placements and are not traded in secondary markets. These investments aim to increase long-term value creation by allowing companies to focus on their operational activities.

The cost of equity is the minimum rate of return that stockholders require from the company for investing in its equity. Understanding the characteristics of equity securities, such as their risk and return profiles, is essential for investors to make informed investment decisions. Additionally, a company's accounting return on equity reflects the total return it earns on shareholders' book equity.
topic_name: Overview of Equity Securities
learning_outcome: describe characteristics of types of equity securities
t

TypeError: can only concatenate str (not "Document") to str

In [83]:
df = pd.read_csv(
    '../output/qa_df_set_a_b_qa.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1)

df["context"] = ""


def similarity_search(query):

    text_field = "text"  # the metadata field that contains our text
    index = "assigmment4"  # the name of the index we created
    embed = OpenAIEmbeddings(
        model='text-embedding-ada-002',
        openai_api_key=OPENAI_API_KEY
    )

    response = vectorstore.similarity_search(
        query,  # our search query
        k=3  # return 3 most relevant docs
    )

    for document in response:
        print(document.page_content)

    return document

for index, row in df.iterrows():

    # Query OpenAI for each row and update the 'Technical Document' colum
    df.at[index, 'context'] = similarity_search(
        query=row['question']
    )

df

Common shares give investors ownership in the company and the opportunity to participate in its performance and decision-making. Preference shares, on the other hand, provide priority in dividend payments over common shareholders. Private equity securities are issued to institutional investors in private placements and are not traded in secondary markets. These investments aim to increase long-term value creation by allowing companies to focus on their operational activities.

The cost of equity is the minimum rate of return that stockholders require from the company for investing in its equity. Understanding the characteristics of equity securities, such as their risk and return profiles, is essential for investors to make informed investment decisions. Additionally, a company's accounting return on equity reflects the total return it earns on shareholders' book equity.
topic_name: Overview of Equity Securities
learning_outcome: describe characteristics of types of equity securities
t

,question,context
0,1. Question: Which of the following statements...,page_content=': Equity securities encompass a ...
1,2. Question: What is the role of financial int...,page_content='\\hline\n\\text{Insurance Compan...
2,3. Question: What is the purpose of security m...,page_content='Rebalancing and reconstitution a...
3,4. Question: What is the difference between pr...,page_content='topic_name: Security Market Inde...
4,5. Question: Why are liquid markets essential ...,page_content='topic_name: Market Organization ...
...,...,...
95,1. Question: Which of the following statements...,page_content=': Equity securities encompass a ...
96,2. Question: What is the role of financial int...,page_content='\\hline\n\\text{Insurance Compan...
97,3. Question: What is the purpose of security m...,page_content='By comprehending the various ind...
98,4. Question: What is the difference between pr...,"page_content='Understanding the value, price r..."


In [84]:
df

,question,context
0,1. Question: Which of the following statements...,page_content=': Equity securities encompass a ...
1,2. Question: What is the role of financial int...,page_content='\\hline\n\\text{Insurance Compan...
2,3. Question: What is the purpose of security m...,page_content='Rebalancing and reconstitution a...
3,4. Question: What is the difference between pr...,page_content='topic_name: Security Market Inde...
4,5. Question: Why are liquid markets essential ...,page_content='topic_name: Market Organization ...
...,...,...
95,1. Question: Which of the following statements...,page_content=': Equity securities encompass a ...
96,2. Question: What is the role of financial int...,page_content='\\hline\n\\text{Insurance Compan...
97,3. Question: What is the purpose of security m...,page_content='By comprehending the various ind...
98,4. Question: What is the difference between pr...,"page_content='Understanding the value, price r..."


In [87]:
df['openai_answer'] = ""
for index, row in df.iterrows():

    # Query OpenAI for each row and update the 'Technical Document' colum
    df.at[index, 'openai_answer'] = generate_openai_response(
        query=row['question'], context=str(row['context'])
    )

df

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


attempting to connect to openai


WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


attempting to connect to openai


WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


...

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


attempting to connect to openai


WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


attempting to connect to openai


,question,context,answers,answerd,openai_answer
0,1. Question: Which of the following statements...,page_content=': Equity securities encompass a ...,,Correct Answer: C. Preference shares are a for...,Correct Answer: C. Preference shares are a for...
1,2. Question: What is the role of financial int...,page_content='\\hline\n\\text{Insurance Compan...,,A. Financial intermediaries create instruments...,A. Financial intermediaries create instruments...
2,3. Question: What is the purpose of security m...,page_content='Rebalancing and reconstitution a...,,The correct answer is: A. Security market inde...,The correct answer is: A. Security market inde...
3,4. Question: What is the difference between pr...,page_content='topic_name: Security Market Inde...,,NaN,D. Total return index reflects both the prices...
4,5. Question: Why are liquid markets essential ...,page_content='topic_name: Market Organization ...,,NaN,C. Liquid markets allow traders to trade when ...
...,...,...,...,...,...
95,1. Question: Which of the following statements...,page_content=': Equity securities encompass a ...,,NaN,Correct Answer: C. Preference shares are a for...
96,2. Question: What is the role of financial int...,page_content='\\hline\n\\text{Insurance Compan...,,NaN,A. Financial intermediaries allow buyers to co...
97,3. Question: What is the purpose of security m...,page_content='By comprehending the various ind...,,NaN,C. To act as benchmarks for actively managed p...
98,4. Question: What is the difference between pr...,"page_content='Understanding the value, price r...",,NaN,C. Price return index reflects both prices of ...


In [88]:
df.to_csv(
    '../output/qa_df_set_a_b_qa_with_openai_response.csv', index=False)